# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

print(sum([sqrt_value for sqrt_value in generator]))

8.382332347441762


In [2]:
limit = 13
generator = square_root_generator(limit)

print([sqrt_value for sqrt_value in generator][-1])

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [4]:
import dlt

In [5]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
                               table_name="people_append",
                               write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.48 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\kerim\github_repo\data-engineering-zoomcamp\homeworks\workshops\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707686731.0311606 is LOADED and contains no failed jobs


In [6]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n People_append table below:")

peoples = conn.sql("SELECT * FROM people_append").df()
display(peoples)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
│ people_append       │
│ people_merge        │
└─────────────────────┘




 People_append table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707686731.0311606,N4M5CKKz0LeiTA
1,2,Person_2,27,City_A,1707686731.0311606,AGeb7aPUz6hccg
2,3,Person_3,28,City_A,1707686731.0311606,WtAYSlf+qPI9yQ
3,4,Person_4,29,City_A,1707686731.0311606,KVPLv9tQ2PrIMA
4,5,Person_5,30,City_A,1707686731.0311606,hN1IhxZiXxndUA


In [7]:
peoples.age.sum()

140

In [8]:
info = generators_pipeline.run(people_2(),
                               table_name="people_append",
                               write_disposition="append")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.48 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\kerim\github_repo\data-engineering-zoomcamp\homeworks\workshops\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707686732.4398925 is LOADED and contains no failed jobs


In [9]:
peoples = conn.sql("SELECT * FROM people_append").df()
display(peoples)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707686731.0311606,N4M5CKKz0LeiTA,None
1,2,Person_2,27,City_A,1707686731.0311606,AGeb7aPUz6hccg,None
2,3,Person_3,28,City_A,1707686731.0311606,WtAYSlf+qPI9yQ,None
3,4,Person_4,29,City_A,1707686731.0311606,KVPLv9tQ2PrIMA,None
4,5,Person_5,30,City_A,1707686731.0311606,hN1IhxZiXxndUA,None
5,3,Person_3,33,City_B,1707686732.4398925,gT5fXhLoWFA/5w,Job_3
6,4,Person_4,34,City_B,1707686732.4398925,IGVk8VxMHgFfZA,Job_4
7,5,Person_5,35,City_B,1707686732.4398925,hZ/EOWBoGUntGQ,Job_5
8,6,Person_6,36,City_B,1707686732.4398925,kpMWYM0UA2JUDA,Job_6
9,7,Person_7,37,City_B,1707686732.4398925,C5wQ7T0VsjUtFA,Job_7


In [10]:
peoples.age.sum()

353

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [11]:
# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
                               table_name="people_merge",
                               write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)
info = generators_pipeline.run(people_2(),
                               table_name="people_merge",
                               write_disposition="merge",
                               primary_key='id')

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.47 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\kerim\github_repo\data-engineering-zoomcamp\homeworks\workshops\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707686735.6758971 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.62 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\kerim\github_repo\data-engineering-zoomcamp\homeworks\workshops\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707686736.8828583 is LOADED and contains no failed jobs


In [12]:
peoples = conn.sql("SELECT * FROM people_merge").df()
display(peoples)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707686735.6758971,n+Jd8dGuJ8zCJg,None
1,2,Person_2,27,City_A,1707686735.6758971,BpEBNscRIm1skA,None
2,5,Person_5,35,City_B,1707686736.8828583,ULXPSesc1AJLGw,Job_5
3,7,Person_7,37,City_B,1707686736.8828583,pRzNwxi6/496pA,Job_7
4,4,Person_4,34,City_B,1707686736.8828583,Q55dDCCuZyTgqQ,Job_4
5,3,Person_3,33,City_B,1707686736.8828583,uHeXZZRAIKQH8Q,Job_3
6,6,Person_6,36,City_B,1707686736.8828583,fTEO3HTKnCDblw,Job_6
7,8,Person_8,38,City_B,1707686736.8828583,NyhzkpnwyzcboA,Job_8


In [13]:
peoples.age.sum()

266